Example workflow to setup an FTU using APIs, generating executable codes for various experimental designs, and generating code for infering operators from data

In [40]:
import numpy as np
import sympy
from IPython.display import display, Math, Latex
#sympy.init_printing(use_latex='mathjax')
import json
from ftuutils import phsutils

Load the positions at which cells exist

Here we construct a graph based on Delaunay triangulation with constraints on maximum edge lengths. 
In cases where nodes and their interconnections are known, construct a Networkx graph and a FTUGraph can be created from the Networkx graph

In [41]:
from ftuutils.base import FTUGraph 
g = FTUGraph()
nxg = g.createBaseGraph()
#0 is the boundary node
g.createInteriorNodes([1,2,3],nxg)
#Create edges and set the weights for edges on the default network
nxg = g.createEdge(1,2,nxg,g.getDefaultNetworkID(),1.0)
nxg = g.createEdge(1,3,nxg,g.getDefaultNetworkID(),1.0)
#Set the node PHS descriptor
celltypes = {1:"HBV",2:"HBV",3:"HBV"}
g.setCellType(celltypes,nxg)

Given a graph describing the cellular interconnections and a set of nodes that exchange energy (input nodes), we can start constructing an FTU (specifically the discrete part of it).

The example below uses a modified form of AlievPanfilov Excitation Contraction model that has membrane voltage and active tension as state variables. The model also has an approximate Hamiltonian description.

In [42]:
#Create a dictionary to store connection information
phsdata = {}
#FTUGraph type assigns a network id, we will use this network for membrane current exchange
defaultNetworkId = 1
#Specify for each PHS class, for each input component the network on which it connects
phsdata = phsutils.connect(phsdata , 'HBV','u1',defaultNetworkId) #Connection on u - flow
phsdata = phsutils.connect(phsdata , 'HBV','u3',defaultNetworkId) #Connection on u - flow

Boundary connections are specified as below. As a convention, boundary networks are negatively numbered.

Nodes that receive external input are specified as below - these are boundary connections and the network number is negative.
All external inputs with the same network number share the same input variable.


If different input variables are required for each  external input, provide different network numbers for the nodes this may be useful for much finer control of inputs.

In this example we assume that all the input nodes are excited by the same stimulus.

In [43]:
phsdata = phsutils.addExternalInput(phsdata,1,'u1',-2)
phsdata = phsutils.addExternalInput(phsdata,1,'u3',-3)

#Indicate which networks are dissipative and add the information to the phsdata dictionary
networkDissipation = {defaultNetworkId:False}
networkNames = {defaultNetworkId:"ucap",-2:"ubarin",-3:"ubarout"}
#The dictionary keys networkNames and networkDissipation are keywords for composition logic and must be adhered
phsdata["networkNames"] = networkNames
phsdata["networkDissipation"] = networkDissipation


Provide Cell type descriptions, these can be constructed by hand, by FTUWeaver or through libbondgraph api.

Below we create a single celltype with the name `APN`

In [44]:
#phsval = r'{"Hderivatives":{"cols":1,"elements":["\\frac{q}{C}","\\frac{v}{L}"],"rows":2},"hamiltonian":"(1/2)*(1/C)*q**2 + (1/2)*(1/L)*p_1**2","hamiltonianLatex":"\\left(\\frac{1}{2}\\right) \\frac{v**2}{L} + \\left(\\frac{1}{2}\\right) \\frac{q**2}{C}","parameter_values":{"C":{"units":"L/mPa","value":"1.125281e-07"},"L":{"units":"kg/m^4","value":"66650"},"r":{"units":"kg*m^-4*s^-1","value":"3999000"},"u":{"units":"Pa","value":"10000"}},"portHamiltonianMatrices":{"matB":{"cols":1,"elements":["0","-1"],"rows":2},"matE":{"cols":2,"elements":["1","0","0","1"],"rows":2},"matJ":{"cols":2,"elements":["0","1","-1","0"],"rows":2},"matQ":{"cols":2,"elements":["1/C","0","0","1/L"],"rows":2},"matR":{"cols":2,"elements":["0","0","0","r"],"rows":2},"u":{"cols":1,"elements":["u"],"rows":1},"u_ispotential":{"cols":1,"elements":[true],"rows":1},"u_orientation":{"cols":1,"elements":[true],"rows":1},"u_split":{"cols":1,"elements":[1],"rows":1},"u_connect2boundary":{"cols":1,"elements":[false],"rows":1}},"stateVector":{"cols":1,"elements":["q","v"],"rows":2},"state_values":{"v":{"units":"Pa*s","value":"0"},"q":{"units":"m^3","value":"0"}},"success":true}'
phsval = r'{"Hderivatives":{"cols":1,"elements":["\\frac{q}{C}","\\frac{p1}{L1}","\\frac{p2}{L2}"],"rows":3},"hamiltonian":"(1/2)*(1/C)*q_0**2 + (1/2)*(1/L_1)*p_1**2 + (1/2)*(1/L_2)*p_2**2","hamiltonianLatex":"\\left(\\frac{1}{2}\\right) \\frac{p1^2}{L1} + \\left(\\frac{1}{2}\\right) \\frac{p2^2}{L2} + \\left(\\frac{1}{2}\\right) \\frac{q^2}{C}","parameter_values":{"C":{"units":"L/mPa","value":"1.125281e-07"},"L1":{"units":"kg/m^4","value":"66650"},"L2":{"units":"kg/m^4","value":"66650"},"r1":{"units":"kg*m^-4*s^-1","value":"3999000"},"r2":{"units":"kg*m^-4*s^-1","value":"3999000"},"u_of_u_1":{"units":"Pa","value":"10000"},"u3":{"units":"Pa","value":"5000"}},"portHamiltonianMatrices":{"matB":{"cols":2,"elements":["0","0","-1","0","0","1"],"rows":3},"matE":{"cols":3,"elements":["1","0","0","0","1","0","0","0","1"],"rows":3},"matJ":{"cols":3,"elements":["0","1","-1","-1","0","0","1","0","0"],"rows":3},"matQ":{"cols":3,"elements":["C**-1","0","0","0","L1**-1","0","0","0","L2**-1"],"rows":3},"matR":{"cols":3,"elements":["0","0","0","0","r1","0","0","0","r2"],"rows":3},"u":{"cols":1,"elements":["u1","u3"],"rows":2},"u_ispotential":{"cols":1,"elements":[true,true],"rows":2},"u_orientation":{"cols":1,"elements":[true,false],"rows":2},"u_split":{"cols":1,"elements":[1,1],"rows":2},"u_connect2boundary":{"cols":1,"elements":[false,false],"rows":2}},"stateVector":{"cols":1,"elements":["q","p1","p2"],"rows":3},"state_values":{"p1":{"units":"Pa*s","value":"0"},"p2":{"units":"Pa*s","value":"0"},"q":{"units":"m^3","value":"0"}},"success":true}'
phstypes = {'HBV':json.loads(phsval)}

With the above information and a graph with appropriate node and edge attributes, we can compose a FTU.

In [45]:
#Call the FTU composition logic to create a FTU with above information

#composer = g.composeCompositePHS(G,phstypes,phsdata)

#The above call will create a composite phs, whose parameters are substituted in the final
#expression. Use this approach when the PHS parameters will not be changed to explored in the experiments
#When experiments with differing phs parameters need to created, the composite PHS
#can be created such that the parameters are not substituted at build time but resolved at runtime
#For such approaches use
composer = g.composeCompositePHS(nxg,phstypes,phsdata,substituteParameters=False)

In [52]:
composer.Cmatrix
print(sympy.latex(composer.Cmatrix))

\left[\begin{matrix}0 & 0 & -1.0 & 0 & -1.0 & 0\\0 & 0 & 0 & -1.0 & 0 & -1.0\\1.0 & 0 & 0 & 0 & 0 & 0\\0 & 1.0 & 0 & 0 & 0 & 0\\1.0 & 0 & 0 & 0 & 0 & 0\\0 & 1.0 & 0 & 0 & 0 & 0\end{matrix}\right]


In [46]:
phsmat = composer.compositePHS

In [47]:
stateVec = sympy.Matrix(composer.stateVec)
ucapVec = sympy.Matrix([f"u_{s}" for s in composer.stateVec])
Ccap = composer.uyConnectionMatrix
Delx = composer.Qcap * stateVec  # Potential
# Since E^-1 can be expensive, we will scale by the rate diagonal value of E for that component
Einv = sympy.eye(composer.Ecap.shape[0])
for i in range(composer.Ecap.shape[0]):
    Einv[i, i] = 1 / composer.Ecap[i, i]
JRQx = (composer.Jcap - composer.Rcap) * Delx
interioru = composer.Bcap * Ccap * (composer.Bcap.T) * ucapVec
exterioru = composer.Bdas * sympy.Matrix(composer.uVecSymbols).T
rhs = sympy.SparseMatrix(Einv * (JRQx - interioru + exterioru))
rhs

Matrix([
[                                    -p2_1/L2_1 + p1_1/L1_1],
[-u1_2 - 1.0*u_p1_2 - 1.0*u_p1_3 - p1_1*r1_1/L1_1 - q_1/C_1],
[ u3_3 - 1.0*u_p2_2 - 1.0*u_p2_3 - p2_1*r2_1/L2_1 + q_1/C_1],
[                                    -p2_2/L2_2 + p1_2/L1_2],
[                     1.0*u_p1_1 - p1_2*r1_2/L1_2 - q_2/C_2],
[                     1.0*u_p2_1 - p2_2*r2_2/L2_2 + q_2/C_2],
[                                    -p2_3/L2_3 + p1_3/L1_3],
[                     1.0*u_p1_1 - p1_3*r1_3/L1_3 - q_3/C_3],
[                     1.0*u_p2_1 - p2_3*r2_3/L2_3 + q_3/C_3]])

In [53]:
print(ucapVec)

Matrix([[u_q_1], [u_p1_1], [u_p2_1], [u_q_2], [u_p1_2], [u_p2_2], [u_q_3], [u_p1_3], [u_p2_3]])


In [48]:
composer.Bdas

Matrix([
[ 0, 0],
[-1, 0],
[ 0, 1],
[ 0, 0],
[ 0, 0],
[ 0, 0],
[ 0, 0],
[ 0, 0],
[ 0, 0]])

In [49]:
composer.Bcap

Matrix([
[0, 0, 0, 0, 0, 0],
[1, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0],
[0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 1]])

If composition was successful, executable python code that simulates the FTU dynamics can be generated. This is the `Full Order Model`

Use the following snippet to get this code and save to disk to execute.
`FOM_python_code = composer.exportAsPython()`

Users are expected to setup the inputs i.e. their magnitude, time and state dependence. The generated code is appropriately commented to help identify these inputs and access time and state variables.

Determining reduced order models or data driven operators requires the generation of large amount of data covering the input and parameter space of interest (we call this experimental design).
To enable this, the composer can export code suitable for a experimental design.
This is acheived as follows.

The generated python code is similar to the python executable code, but is now encapsulated within a class.
The executable code should be used to examine the model, test and also decide in specifiying experiments.
If experiments are created, this code is also saved as `FTUStepper.py` in the experiment directory.

In [50]:
odestepper = composer.exportAsODEStepper()